# Title
[]()

In [2]:

import sys
sys.path.append(r"/home/silvhua/custom_python")
sys.path.append('/home/silvhua/repositories/GHL-chat/src/app')
sys.path.append('/home/silvhua/repositories/GHL-chat/src')
from silvhua import *
from ghl_requests import refresh_token

In [ ]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 100)
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Original

In [5]:
# import pytz # Not included in Lambda by default
import sys
import requests
from datetime import datetime, timedelta
import json
from urllib.parse import urlencode
import boto3
import random

def ghl_request(
        contactId, endpoint='createTask', text=None, payload=None, location='SamLab', 
        path_param=None
        ):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID OR locationId if endpoint is 'getWorkflow'.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'workflow', 'getWorkflow', \
    'createNote', 'send_message', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.
    - path_param (str): Additional path parameter for the request.

    Example payload for sendMessage endpoint:
        
            payload = {
                "type": "Email",
                "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
                "subject": "Testing GHL API with Python threadId",
                "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
                "threadId": 'Y0ecBIIPiHa7bpZr616G'
            }

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'
    if payload:
        print(f'input payload: {payload}')
    try:
        if endpoint == 'getContact':
            endpoint_url = f'contacts/{contactId}'
            request_type = 'GET'
            payload = None
        elif endpoint == 'createTask':
            endpoint_url = f'contacts/{contactId}/tasks'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['title'] = f'Send message to contact {contactId}'
                payload['body'] = text if text else f"Test task via GHL API at {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')} Pacific time"
            payload['dueDate'] = payload[3] if len(payload) > 3 else datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
            payload['completed'] = False
        elif endpoint == 'workflow':
            endpoint_url = f'contacts/{contactId}/workflow/{path_param}'
            request_type = 'POST'
            payload = {}
            payload['eventStartTime'] = (datetime.utcnow() + timedelta(minutes=random.randint(2, 10))).strftime('%Y-%m-%dT%H:%M:%S+00:00')
        elif endpoint == 'getWorkflow':
            endpoint_url = r'workflows/'
            request_type = 'GET'
            params = {'locationId': contactId}
        elif endpoint == 'createNote':
            endpoint_url = f'contacts/{contactId}/notes'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['body'] = (f"Reply to SMS (contactID {contactId}) {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}" if text==None else text)
                payload['userId'] = contactId
        elif endpoint == 'sendMessage':
            endpoint_url = f'conversations/messages'
            request_type = 'POST'
            payload["contactId"] = contactId
            if payload['type'] == 'Email':
                payload['html'] = payload['message']
        elif endpoint == 'getEmailHistory':
            endpoint_url = f'conversations/search?contactId={contactId}'
            request_type = 'GET'
            payload = None
        else:
            raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'sendMessage', and 'getEmailHistory'.")

        url = f'{url_root}{endpoint_url}'
        try:
            s3 = boto3.client('s3')
            response = s3.get_object(Bucket='ownitfit-silvhua', Key='auth_token_response.json')
            token = json.loads(response['Body'].read().decode('utf-8'))[location]
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            message = f'Error in line {lineno} of {filename}: {str(error)}'
            print(message)
            return message

        headers = {
            "Authorization": f"Bearer {token['access_token']}",
            "Version": "2021-04-15",
            "Content-Type": "application/json",
            "Accept": "application/json"
        }


        if request_type == 'POST':
            response = requests.post(
                url, headers=headers, 
                json=payload if payload else None
            )
        elif request_type == 'GET':
            response = requests.get(
                url, headers=headers, 
                json=payload if payload else None,
                # params=params if params else None
            )
        else:
            raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

        print(f'Status code {response.status_code}: {response.reason}')
        data = response.json()
        data['status_code'] = response.status_code
        data['response_reason'] = response.reason
        try:
            if endpoint == 'getEmailHistory':
                email_timestamp = data['conversations'][0]['dateUpdated']/1000
                utc_time = datetime.utcfromtimestamp(email_timestamp)
                pacific_time = utc_time.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('US/Pacific'))
                email_timestamp_str = pacific_time.strftime('%Y-%m-%d %H:%M:%S')

                print(f'Last email sent: {email_timestamp_str} Pacific time')
                data['emailTimestamp_pacific'] = email_timestamp_str
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print(f'Error in line {lineno} of {filename}: {str(error)}')
            return '[Chatbot response]'
        return data
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print(f'Error in line {lineno} of {filename}: {str(error)}')
        return '[Chatbot response]'

# set up

In [17]:
token_response = refresh_token('../src/app/private')

Tokens retrieved from S3.
Tokens saved to S3.


In [4]:
response_dict = dict()

In [3]:
csv_filename = 'BQ Lead Generation tracker - Brian Quinlan - Lead Tracker.csv'
csv_path = '../data/raw'
leads_df = load_csv(csv_filename, csv_path)

Dataframe shape:  (1554, 13)
DataFrame columns: ['Unnamed: 0', 'Date Recorded', 'Social Media', 'Name', 'IG', 'Email', 'Phone Number', 'Lead Origin', 'Booked Schedule', 'Price Point', 'Status', 'Notes', 'Unnamed: 12']
	Time completed: 2024-01-02 14:27:38.275843


In [5]:

auth_info = load_json('auth_token_response.json', '../src/app/private')
locationId = auth_info['CoachMcloone']['locationId']

In [ ]:
leads_df.loc[1533, 'Name']

In [7]:
leads_df.index

RangeIndex(start=0, stop=1554, step=1)

# `process_leads_csv`

In [7]:
def process_leads_csv(csv_filename, csv_path):
    df = load_csv(csv_filename, csv_path)
    # Update the index to start at 2 instead of zero
    df.index = df.index + 2
    print(f'Index updated to start at 2 isntead of 0.')
    return df

csv_filename = 'BQ Lead Generation tracker - Brian Quinlan - Lead Tracker.csv'
csv_path = '../data/raw'
leads_df = process_leads_csv(csv_filename, csv_path)

Dataframe shape:  (1554, 13)
DataFrame columns: ['Unnamed: 0', 'Date Recorded', 'Social Media', 'Name', 'IG', 'Email', 'Phone Number', 'Lead Origin', 'Booked Schedule', 'Price Point', 'Status', 'Notes', 'Unnamed: 12']
	Time completed: 2024-01-02 15:06:17.739054
Index updated to start at 2 isntead of 0.


# Iteration 1

In [19]:
# import pytz # Not included in Lambda by default
import sys
import requests
from datetime import datetime, timedelta
import json
from urllib.parse import urlencode
import boto3
import random

def ghl_request(
        contactId, endpoint='createTask', text=None, payload=None, location='SamLab', 
        path_param=None, locationId=None
        ):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID OR locationId if endpoint is 'getWorkflow'.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'workflow', 'getWorkflow', \
    'createNote', 'send_message', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.
    - path_param (str): Additional path parameter for the request.
    - locationId: locationId for getContacts endpoint.

    Example payload for sendMessage endpoint:
        
            payload = {
                "type": "Email",
                "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
                "subject": "Testing GHL API with Python threadId",
                "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
                "threadId": 'Y0ecBIIPiHa7bpZr616G'
            }

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'
    if payload:
        print(f'input payload: {payload}')
    try:
        if endpoint == 'getContact':
            endpoint_url = f'contacts/{contactId}'
            request_type = 'GET'
            payload = None
        elif endpoint == 'createTask':
            endpoint_url = f'contacts/{contactId}/tasks'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['title'] = f'Send message to contact {contactId}'
                payload['body'] = text if text else f"Test task via GHL API at {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')} Pacific time"
            payload['dueDate'] = payload[3] if len(payload) > 3 else datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
            payload['completed'] = False
        elif endpoint == 'workflow':
            endpoint_url = f'contacts/{contactId}/workflow/{path_param}'
            request_type = 'POST'
            payload = {}
            payload['eventStartTime'] = (datetime.utcnow() + timedelta(minutes=random.randint(2, 10))).strftime('%Y-%m-%dT%H:%M:%S+00:00')
        elif endpoint == 'getWorkflow': ### 
            endpoint_url = r'workflows/'
            request_type = 'GET'
            params = {'locationId': contactId if contactId else locationId}
        elif endpoint == 'createNote':
            endpoint_url = f'contacts/{contactId}/notes'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['body'] = (f"Reply to SMS (contactID {contactId}) {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}" if text==None else text)
                payload['userId'] = contactId
        elif endpoint == 'sendMessage':
            endpoint_url = f'conversations/messages'
            request_type = 'POST'
            payload["contactId"] = contactId
            if payload['type'] == 'Email':
                payload['html'] = payload['message']
        elif endpoint == 'getEmailHistory':
            endpoint_url = f'conversations/search?contactId={contactId}'
            request_type = 'GET'
            payload = None
        elif endpoint == 'getContacts': ### 
            endpoint_url = f'conversations/contacts'
            request_type = 'GET'
            payload = None
            params = {
                'locationId': locationId,
                'query': contactId
            }
        else:
            raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'sendMessage', and 'getEmailHistory'.")

        url = f'{url_root}{endpoint_url}'
        try:
            s3 = boto3.client('s3')
            response = s3.get_object(Bucket='ownitfit-silvhua', Key='auth_token_response.json')
            token = json.loads(response['Body'].read().decode('utf-8'))[location]
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            message = f'Error in line {lineno} of {filename}: {str(error)}'
            print(message)
            return message

        headers = {
            "Authorization": f"Bearer {token['access_token']}",
            "Version": "2021-04-15",
            "Content-Type": "application/json",
            "Accept": "application/json"
        }


        if request_type == 'POST':
            response = requests.post(
                url, headers=headers, 
                json=payload if payload else None
            )
        elif request_type == 'GET':
            response = requests.get(
                url, headers=headers, 
                json=payload if payload else None,
                params=params if params else None
            )
        else:
            raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

        print(f'Status code {response.status_code}: {response.reason}')
        data = response.json()
        data['status_code'] = response.status_code
        data['response_reason'] = response.reason
        try:
            if endpoint == 'getEmailHistory':
                email_timestamp = data['conversations'][0]['dateUpdated']/1000
                utc_time = datetime.utcfromtimestamp(email_timestamp)
                pacific_time = utc_time.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('US/Pacific'))
                email_timestamp_str = pacific_time.strftime('%Y-%m-%d %H:%M:%S')

                print(f'Last email sent: {email_timestamp_str} Pacific time')
                data['emailTimestamp_pacific'] = email_timestamp_str
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print(f'Error in line {lineno} of {filename}: {str(error)}')
            return '[Chatbot response]'
        return data
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print(f'Error in line {lineno} of {filename}: {str(error)}')
        return '[Chatbot response]'

iteration = 1
query_string = leads_df.loc[1533, 'Name']

response_dict[iteration] = ghl_request(
    query_string, endpoint='getContacts', text=None, payload=None, location='CoachMcloone', 
    path_param=None, locationId=locationId
)

Status code 401: Unauthorized


In [20]:
response_dict[iteration]

{'statusCode': 401,
 'message': 'Invalid token: access token is invalid',
 'error': 'Unauthorized',
 'status_code': 401,
 'response_reason': 'Unauthorized'}

# Iteration 2 after kernel refresh

In [8]:
# import pytz # Not included in Lambda by default
import sys
import requests
from datetime import datetime, timedelta
import json
from urllib.parse import urlencode
import boto3
import random




def refresh_token(location='CoachMcloone', token_file_path = 'app/private'):
    """
    Refreshes the authentication token. 
    Parameters:
    - location (str): The location of the business in Pascal case, i.e. 'SamLab' or 'CoachMcloone'.
    - token_file_path (str): The path to the token file relative to the directory of the lambda function

    This function does the following:
    1. Reads the application configuration from a local file.
    2. Attempts to load the authentication tokens from a local file.
    3. If local file is not found, it retrieves the tokens from an S3 bucket.
    4. Checks if the 'SamLab' token is present in the loaded tokens. If not, it returns an error.
    5. Prepares data for the token refresh request, including client_id, client_secret, grant_type, refresh_token, user_type, and redirect_uri from the application configuration and SamLab token.
    6. Sends a POST request to the Lead Connector API to refresh the token.
    7. If the token refresh request is successful, it updates the 'SamLab' token in the loaded tokens with the response and tries to save it to a local file.
    8. If local save fails, it tries to save the tokens to the same S3 bucket.
    9. If S3 save also fails, it prints the error details.
    10. Returns the response of the token refresh request.

    Returns:
        dict: A dictionary with 'statusCode', 'body', and optionally 'response' keys. 'statusCode' is 200 if the token refresh request is successful, and 500 otherwise. 'body' contains the response of the token refresh request if it is successful, and an error message otherwise. 'response' is present only if the token refresh request fails, and contains the response of the failed request.
    """

    filename = 'auth_token_response_cicd.json'
    # filename = 'auth_token_response.json' # Original
    config_file_name = 'config.json'
    # config_file_name = 'config_cicid_app.json' # cicd

    with open(f'{token_file_path}/{config_file_name}') as config_file:
        appConfig = json.load(config_file)
    s3 = boto3.client('s3')
    response = s3.get_object(Bucket='ownitfit-silvhua', Key=filename)
    tokens = json.loads(response['Body'].read().decode('utf-8'))
    print(f'Tokens retrieved from S3.')
    if location in tokens:
        previous_token = tokens[location]
    else:
        return {
            'statusCode': 500,
            'body': json.dumps({"error": "SamLab token not found in token_file."})
        }

    data = {
        'client_id': appConfig["clientId"],
        'client_secret': appConfig["clientSecret"],
        'grant_type': 'refresh_token',
        'refresh_token': previous_token["refresh_token"],
        'user_type': 'Location',
        # 'redirect_uri': 'https://6r8pb7q836.execute-api.us-west-2.amazonaws.com/oauth/callback',
        'redirect_uri': 'http://localhost:3000/oauth/callback'
    }

    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    response = requests.post('https://services.leadconnectorhq.com/oauth/token', data=urlencode(data), headers=headers)

    if response.status_code == 200:
        tokens[location] = response.json()
        # pprint(f'Tokens: {tokens["SamLab"]}')
        try:
            # Save tokens to S3
            s3 = boto3.client('s3')
            s3.put_object(
                Body=json.dumps(tokens), 
                Bucket='ownitfit-silvhua', Key=filename
                )
            print(f'Tokens saved to S3.')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print(f"Unable to save tokens to S3. Error in line {lineno} of {filename}: {str(error)}")
        return {
            'statusCode': 200,
            'body': json.dumps(response.json())
        }
    else:
        return {
            'statusCode': 500,
            'body': json.dumps({"error": f"Failed to fetch access token: {response.reason}",}),
            'response': json.dumps(response.json())
        }

def ghl_request(
        contactId, endpoint='createTask', text=None, payload=None, location='SamLab', 
        path_param=None, locationId=None
        ):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID OR locationId if endpoint is 'getWorkflow'.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'workflow', 'getWorkflow', \
    'createNote', 'send_message', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.
    - path_param (str): Additional path parameter for the request.
    - locationId: locationId for getContacts endpoint.

    Example payload for sendMessage endpoint:
        
            payload = {
                "type": "Email",
                "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
                "subject": "Testing GHL API with Python threadId",
                "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
                "threadId": 'Y0ecBIIPiHa7bpZr616G'
            }

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'
    if payload:
        print(f'input payload: {payload}')
    try:
        if endpoint == 'getContact':
            endpoint_url = f'contacts/{contactId}'
            request_type = 'GET'
            payload = None
        elif endpoint == 'createTask':
            endpoint_url = f'contacts/{contactId}/tasks'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['title'] = f'Send message to contact {contactId}'
                payload['body'] = text if text else f"Test task via GHL API at {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')} Pacific time"
            payload['dueDate'] = payload[3] if len(payload) > 3 else datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
            payload['completed'] = False
        elif endpoint == 'workflow':
            endpoint_url = f'contacts/{contactId}/workflow/{path_param}'
            request_type = 'POST'
            payload = {}
            payload['eventStartTime'] = (datetime.utcnow() + timedelta(minutes=random.randint(2, 10))).strftime('%Y-%m-%dT%H:%M:%S+00:00')
        elif endpoint == 'getWorkflow': ### 
            endpoint_url = r'workflows/'
            request_type = 'GET'
            params = {'locationId': contactId if contactId else locationId}
        elif endpoint == 'createNote':
            endpoint_url = f'contacts/{contactId}/notes'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['body'] = (f"Reply to SMS (contactID {contactId}) {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}" if text==None else text)
                payload['userId'] = contactId
        elif endpoint == 'sendMessage':
            endpoint_url = f'conversations/messages'
            request_type = 'POST'
            payload["contactId"] = contactId
            if payload['type'] == 'Email':
                payload['html'] = payload['message']
        elif endpoint == 'getEmailHistory':
            endpoint_url = f'conversations/search?contactId={contactId}'
            request_type = 'GET'
            payload = None
        elif endpoint == 'getContacts': ### 
            endpoint_url = f'conversations/contacts'
            request_type = 'GET'
            payload = None
            params = {
                'locationId': locationId,
                'query': contactId
            }
        else:
            raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'sendMessage', and 'getEmailHistory'.")

        url = f'{url_root}{endpoint_url}'
        try:
            s3 = boto3.client('s3')
            response = s3.get_object(Bucket='ownitfit-silvhua', Key='auth_token_response.json')
            token = json.loads(response['Body'].read().decode('utf-8'))[location]
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            message = f'Error in line {lineno} of {filename}: {str(error)}'
            print(message)
            return message

        headers = {
            "Authorization": f"Bearer {token['access_token']}",
            "Version": "2021-04-15",
            "Content-Type": "application/json",
            "Accept": "application/json"
        }


        if request_type == 'POST':
            response = requests.post(
                url, headers=headers, 
                json=payload if payload else None
            )
        elif request_type == 'GET':
            response = requests.get(
                url, headers=headers, 
                json=payload if payload else None,
                params=params if params else None
            )
        else:
            raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

        print(f'Status code {response.status_code}: {response.reason}')
        data = response.json()
        data['status_code'] = response.status_code
        data['response_reason'] = response.reason
        try:
            if endpoint == 'getEmailHistory':
                email_timestamp = data['conversations'][0]['dateUpdated']/1000
                utc_time = datetime.utcfromtimestamp(email_timestamp)
                pacific_time = utc_time.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('US/Pacific'))
                email_timestamp_str = pacific_time.strftime('%Y-%m-%d %H:%M:%S')

                print(f'Last email sent: {email_timestamp_str} Pacific time')
                data['emailTimestamp_pacific'] = email_timestamp_str
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print(f'Error in line {lineno} of {filename}: {str(error)}')
            return '[Chatbot response]'
        return data
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print(f'Error in line {lineno} of {filename}: {str(error)}')
        return '[Chatbot response]'

iteration = 1
query_string = leads_df.loc[1533, 'Name']

token_response = refresh_token('CoachMcloone', '../src/app/private')

# response_dict[iteration] = ghl_request(
#     query_string, endpoint='getContacts', text=None, payload=None, location='CoachMcloone', 
#     path_param=None, locationId=locationId
# )

Tokens retrieved from S3.
Tokens saved to S3.


In [10]:
response_dict[iteration] = ghl_request(
    query_string, endpoint='getContacts', text=None, payload=None, location='CoachMcloone', 
    path_param=None, locationId=locationId
)

Status code 401: Unauthorized


In [19]:
response_dict[iteration]

{'statusCode': 401,
 'message': 'Invalid token: access token is invalid',
 'error': 'Unauthorized',
 'status_code': 401,
 'response_reason': 'Unauthorized'}

## 2.1 `refresh_token`; failed GHL request

In [11]:
# import pytz # Not included in Lambda by default
import sys
import requests
from datetime import datetime, timedelta
import json
from urllib.parse import urlencode
import boto3
import random




def refresh_token(location='CoachMcloone', token_file_path = 'app/private'):
    """
    Refreshes the authentication token. 
    Parameters:
    - location (str): The location of the business in Pascal case, i.e. 'SamLab' or 'CoachMcloone'.
    - token_file_path (str): The path to the token file relative to the directory of the lambda function

    This function does the following:
    1. Reads the application configuration from a local file.
    2. Attempts to load the authentication tokens from a local file.
    3. If local file is not found, it retrieves the tokens from an S3 bucket.
    4. Checks if the 'SamLab' token is present in the loaded tokens. If not, it returns an error.
    5. Prepares data for the token refresh request, including client_id, client_secret, grant_type, refresh_token, user_type, and redirect_uri from the application configuration and SamLab token.
    6. Sends a POST request to the Lead Connector API to refresh the token.
    7. If the token refresh request is successful, it updates the 'SamLab' token in the loaded tokens with the response and tries to save it to a local file.
    8. If local save fails, it tries to save the tokens to the same S3 bucket.
    9. If S3 save also fails, it prints the error details.
    10. Returns the response of the token refresh request.

    Returns:
        dict: A dictionary with 'statusCode', 'body', and optionally 'response' keys. 'statusCode' is 200 if the token refresh request is successful, and 500 otherwise. 'body' contains the response of the token refresh request if it is successful, and an error message otherwise. 'response' is present only if the token refresh request fails, and contains the response of the failed request.
    """

    filename = 'auth_token_response_cicd.json'
    # filename = 'auth_token_response.json' # Original
    config_file_name = 'config.json'
    # config_file_name = 'config_cicid_app.json' # cicd

    with open(f'{token_file_path}/{config_file_name}') as config_file:
        appConfig = json.load(config_file)
    s3 = boto3.client('s3')
    response = s3.get_object(Bucket='ownitfit-silvhua', Key=filename)
    tokens = json.loads(response['Body'].read().decode('utf-8'))
    if location in tokens:
        previous_token = tokens[location]
        print(f'Tokens retrieved from S3 for {location}.')
    else:
        return {
            'statusCode': 500,
            'body': json.dumps({"error": "SamLab token not found in token_file."})
        }

    data = {
        'client_id': appConfig["clientId"],
        'client_secret': appConfig["clientSecret"],
        'grant_type': 'refresh_token',
        'refresh_token': previous_token["refresh_token"],
        'user_type': 'Location',
        # 'redirect_uri': 'https://6r8pb7q836.execute-api.us-west-2.amazonaws.com/oauth/callback',
        'redirect_uri': 'http://localhost:3000/oauth/callback'
    }

    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    response = requests.post('https://services.leadconnectorhq.com/oauth/token', data=urlencode(data), headers=headers)

    if response.status_code == 200:
        tokens[location] = response.json()
        # pprint(f'Tokens: {tokens["SamLab"]}')
        try:
            # Save tokens to S3
            s3 = boto3.client('s3')
            s3.put_object(
                Body=json.dumps(tokens), 
                Bucket='ownitfit-silvhua', Key=filename
                )
            print(f'Tokens saved to S3 {location}.')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print(f"Unable to save tokens to S3. Error in line {lineno} of {filename}: {str(error)}")
        return {
            'statusCode': 200,
            'body': json.dumps(response.json())
        }
    else:
        return {
            'statusCode': 500,
            'body': json.dumps({"error": f"Failed to fetch access token: {response.reason}",}),
            'response': json.dumps(response.json())
        }

def ghl_request(
        contactId, endpoint='createTask', text=None, payload=None, location='SamLab', 
        path_param=None, locationId=None
        ):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID OR locationId if endpoint is 'getWorkflow'.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'workflow', 'getWorkflow', \
    'createNote', 'send_message', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.
    - path_param (str): Additional path parameter for the request.
    - locationId: locationId for getContacts endpoint.

    Example payload for sendMessage endpoint:
        
            payload = {
                "type": "Email",
                "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
                "subject": "Testing GHL API with Python threadId",
                "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
                "threadId": 'Y0ecBIIPiHa7bpZr616G'
            }

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'
    if payload:
        print(f'input payload: {payload}')
    try:
        if endpoint == 'getContact':
            endpoint_url = f'contacts/{contactId}'
            request_type = 'GET'
            payload = None
        elif endpoint == 'createTask':
            endpoint_url = f'contacts/{contactId}/tasks'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['title'] = f'Send message to contact {contactId}'
                payload['body'] = text if text else f"Test task via GHL API at {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')} Pacific time"
            payload['dueDate'] = payload[3] if len(payload) > 3 else datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
            payload['completed'] = False
        elif endpoint == 'workflow':
            endpoint_url = f'contacts/{contactId}/workflow/{path_param}'
            request_type = 'POST'
            payload = {}
            payload['eventStartTime'] = (datetime.utcnow() + timedelta(minutes=random.randint(2, 10))).strftime('%Y-%m-%dT%H:%M:%S+00:00')
        elif endpoint == 'getWorkflow': ### 
            endpoint_url = r'workflows/'
            request_type = 'GET'
            params = {'locationId': contactId if contactId else locationId}
        elif endpoint == 'createNote':
            endpoint_url = f'contacts/{contactId}/notes'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['body'] = (f"Reply to SMS (contactID {contactId}) {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}" if text==None else text)
                payload['userId'] = contactId
        elif endpoint == 'sendMessage':
            endpoint_url = f'conversations/messages'
            request_type = 'POST'
            payload["contactId"] = contactId
            if payload['type'] == 'Email':
                payload['html'] = payload['message']
        elif endpoint == 'getEmailHistory':
            endpoint_url = f'conversations/search?contactId={contactId}'
            request_type = 'GET'
            payload = None
        elif endpoint == 'getContacts': ### 
            endpoint_url = f'conversations/contacts'
            request_type = 'GET'
            payload = None
            params = {
                'locationId': locationId,
                'query': contactId
            }
        else:
            raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'sendMessage', and 'getEmailHistory'.")

        url = f'{url_root}{endpoint_url}'
        try:
            s3 = boto3.client('s3')
            response = s3.get_object(Bucket='ownitfit-silvhua', Key='auth_token_response.json')
            token = json.loads(response['Body'].read().decode('utf-8'))[location]
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            message = f'Error in line {lineno} of {filename}: {str(error)}'
            print(message)
            return message

        headers = {
            "Authorization": f"Bearer {token['access_token']}",
            "Version": "2021-04-15",
            "Content-Type": "application/json",
            "Accept": "application/json"
        }


        if request_type == 'POST':
            response = requests.post(
                url, headers=headers, 
                json=payload if payload else None
            )
        elif request_type == 'GET':
            response = requests.get(
                url, headers=headers, 
                json=payload if payload else None,
                params=params if params else None
            )
        else:
            raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

        print(f'Status code {response.status_code}: {response.reason}')
        data = response.json()
        data['status_code'] = response.status_code
        data['response_reason'] = response.reason
        try:
            if endpoint == 'getEmailHistory':
                email_timestamp = data['conversations'][0]['dateUpdated']/1000
                utc_time = datetime.utcfromtimestamp(email_timestamp)
                pacific_time = utc_time.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('US/Pacific'))
                email_timestamp_str = pacific_time.strftime('%Y-%m-%d %H:%M:%S')

                print(f'Last email sent: {email_timestamp_str} Pacific time')
                data['emailTimestamp_pacific'] = email_timestamp_str
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print(f'Error in line {lineno} of {filename}: {str(error)}')
            return '[Chatbot response]'
        return data
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print(f'Error in line {lineno} of {filename}: {str(error)}')
        return '[Chatbot response]'

iteration = 1
query_string = leads_df.loc[1533, 'Name']

token_response = refresh_token('CoachMcloone', '../src/app/private')

# response_dict[iteration] = ghl_request(
#     query_string, endpoint='getContacts', text=None, payload=None, location='CoachMcloone', 
#     path_param=None, locationId=locationId
# )

Tokens retrieved from S3 for CoachMcloone.
Tokens saved to S3 CoachMcloone.


In [20]:
iteration = 2.1
query_string = leads_df.loc[1533, 'Name']

# token_response = refresh_token('CoachMcloone', '../src/app/private')

response_dict[iteration] = ghl_request(
    query_string, endpoint='getContacts', text=None, payload=None, location='CoachMcloone', 
    path_param=None, locationId=locationId
)

Status code 401: Unauthorized


## 2.2

In [ ]:
# import pytz # Not included in Lambda by default
import sys
import requests
from datetime import datetime, timedelta
import json
from urllib.parse import urlencode
import boto3
import random




def refresh_token(location='CoachMcloone', token_file_path = 'app/private'):
    """
    Refreshes the authentication token. 
    Parameters:
    - location (str): The location of the business in Pascal case, i.e. 'SamLab' or 'CoachMcloone'.
    - token_file_path (str): The path to the token file relative to the directory of the lambda function

    This function does the following:
    1. Reads the application configuration from a local file.
    2. Attempts to load the authentication tokens from a local file.
    3. If local file is not found, it retrieves the tokens from an S3 bucket.
    4. Checks if the 'SamLab' token is present in the loaded tokens. If not, it returns an error.
    5. Prepares data for the token refresh request, including client_id, client_secret, grant_type, refresh_token, user_type, and redirect_uri from the application configuration and SamLab token.
    6. Sends a POST request to the Lead Connector API to refresh the token.
    7. If the token refresh request is successful, it updates the 'SamLab' token in the loaded tokens with the response and tries to save it to a local file.
    8. If local save fails, it tries to save the tokens to the same S3 bucket.
    9. If S3 save also fails, it prints the error details.
    10. Returns the response of the token refresh request.

    Returns:
        dict: A dictionary with 'statusCode', 'body', and optionally 'response' keys. 'statusCode' is 200 if the token refresh request is successful, and 500 otherwise. 'body' contains the response of the token refresh request if it is successful, and an error message otherwise. 'response' is present only if the token refresh request fails, and contains the response of the failed request.
    """

    filename = 'auth_token_response_cicd.json'
    # filename = 'auth_token_response.json' # Original
    config_file_name = 'config.json'
    # config_file_name = 'config_cicid_app.json' # cicd

    with open(f'{token_file_path}/{config_file_name}') as config_file:
        appConfig = json.load(config_file)
    s3 = boto3.client('s3')
    response = s3.get_object(Bucket='ownitfit-silvhua', Key=filename)
    tokens = json.loads(response['Body'].read().decode('utf-8'))
    if location in tokens:
        previous_token = tokens[location]
        print(f'Tokens retrieved from S3 for {location}.')
    else:
        return {
            'statusCode': 500,
            'body': json.dumps({"error": "SamLab token not found in token_file."})
        }

    data = {
        'client_id': appConfig["clientId"],
        'client_secret': appConfig["clientSecret"],
        'grant_type': 'refresh_token',
        'refresh_token': previous_token["refresh_token"],
        'user_type': 'Location',
        # 'redirect_uri': 'https://6r8pb7q836.execute-api.us-west-2.amazonaws.com/oauth/callback',
        'redirect_uri': 'http://localhost:3000/oauth/callback'
    }

    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    response = requests.post('https://services.leadconnectorhq.com/oauth/token', data=urlencode(data), headers=headers)

    if response.status_code == 200:
        tokens[location] = response.json()
        # pprint(f'Tokens: {tokens["SamLab"]}')
        try:
            # Save tokens to S3
            s3 = boto3.client('s3')
            s3.put_object(
                Body=json.dumps(tokens), 
                Bucket='ownitfit-silvhua', Key=filename
                )
            print(f'Tokens saved to S3 {location}.')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print(f"Unable to save tokens to S3. Error in line {lineno} of {filename}: {str(error)}")
        return {
            'statusCode': 200,
            'body': json.dumps(response.json())
        }
    else:
        return {
            'statusCode': 500,
            'body': json.dumps({"error": f"Failed to fetch access token: {response.reason}",}),
            'response': json.dumps(response.json())
        }

def ghl_request(
        contactId, endpoint='createTask', text=None, payload=None, location='SamLab', 
        path_param=None, locationId=None
        ):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID OR locationId if endpoint is 'getWorkflow'.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'workflow', 'getWorkflow', \
    'createNote', 'send_message', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.
    - path_param (str): Additional path parameter for the request.
    - locationId: locationId for getContacts endpoint.

    Example payload for sendMessage endpoint:
        
            payload = {
                "type": "Email",
                "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
                "subject": "Testing GHL API with Python threadId",
                "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
                "threadId": 'Y0ecBIIPiHa7bpZr616G'
            }

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'
    if payload:
        print(f'input payload: {payload}')
    try:
        if endpoint == 'getContact':
            endpoint_url = f'contacts/{contactId}'
            request_type = 'GET'
            payload = None
        elif endpoint == 'createTask':
            endpoint_url = f'contacts/{contactId}/tasks'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['title'] = f'Send message to contact {contactId}'
                payload['body'] = text if text else f"Test task via GHL API at {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')} Pacific time"
            payload['dueDate'] = payload[3] if len(payload) > 3 else datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
            payload['completed'] = False
        elif endpoint == 'workflow':
            endpoint_url = f'contacts/{contactId}/workflow/{path_param}'
            request_type = 'POST'
            payload = {}
            payload['eventStartTime'] = (datetime.utcnow() + timedelta(minutes=random.randint(2, 10))).strftime('%Y-%m-%dT%H:%M:%S+00:00')
        elif endpoint == 'getWorkflow': ### 
            endpoint_url = r'workflows/'
            request_type = 'GET'
            params = {'locationId': contactId if contactId else locationId}
        elif endpoint == 'createNote':
            endpoint_url = f'contacts/{contactId}/notes'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['body'] = (f"Reply to SMS (contactID {contactId}) {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}" if text==None else text)
                payload['userId'] = contactId
        elif endpoint == 'sendMessage':
            endpoint_url = f'conversations/messages'
            request_type = 'POST'
            payload["contactId"] = contactId
            if payload['type'] == 'Email':
                payload['html'] = payload['message']
        elif endpoint == 'getEmailHistory':
            endpoint_url = f'conversations/search?contactId={contactId}'
            request_type = 'GET'
            payload = None
        elif endpoint == 'getContacts': ### 
            endpoint_url = f'conversations/contacts'
            request_type = 'GET'
            payload = None
            params = {
                'locationId': locationId,
                'query': contactId
            }
        else:
            raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'sendMessage', and 'getEmailHistory'.")

        url = f'{url_root}{endpoint_url}'
        token_filename = 'auth_token_response_cicd.json'
        try:
            s3 = boto3.client('s3')
            response = s3.get_object(Bucket='ownitfit-silvhua', Key=token_filename)
            token = json.loads(response['Body'].read().decode('utf-8'))[location]
            from pprint import pprint
            pprint(f'Token from S3: \n\n{token}')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            message = f'Error in line {lineno} of {filename}: {str(error)}'
            print(message)
            return message

        headers = {
            "Authorization": f"Bearer {token['access_token']}",
            "Version": "2021-07-28",
            "Content-Type": "application/json",
            "Accept": "application/json"
        }


        if request_type == 'POST':
            response = requests.post(
                url, headers=headers, 
                json=payload if payload else None
            )
        elif request_type == 'GET':
            response = requests.get(
                url, headers=headers, 
                json=payload if payload else None,
                params=params if params else None
            )
        else:
            raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

        print(f'Status code {response.status_code}: {response.reason}')
        data = response.json()
        data['status_code'] = response.status_code
        data['response_reason'] = response.reason
        try:
            if endpoint == 'getEmailHistory':
                email_timestamp = data['conversations'][0]['dateUpdated']/1000
                utc_time = datetime.utcfromtimestamp(email_timestamp)
                pacific_time = utc_time.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('US/Pacific'))
                email_timestamp_str = pacific_time.strftime('%Y-%m-%d %H:%M:%S')

                print(f'Last email sent: {email_timestamp_str} Pacific time')
                data['emailTimestamp_pacific'] = email_timestamp_str
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print(f'Error in line {lineno} of {filename}: {str(error)}')
            return '[Chatbot response]'
        return data
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print(f'Error in line {lineno} of {filename}: {str(error)}')
        return '[Chatbot response]'

iteration = 2.2
query_string = leads_df.loc[1533, 'Name']

# token_response = refresh_token('CoachMcloone', '../src/app/private')

response_dict[iteration] = ghl_request(
    query_string, endpoint='getContacts', text=None, payload=None, location='CoachMcloone', 
    path_param=None, locationId=locationId
)

In [26]:
response_dict[iteration]

{'error': 'Conversation with id contacts not found',
 'status_code': 400,
 'response_reason': 'Bad Request'}

## 2.3

In [27]:
# import pytz # Not included in Lambda by default
import sys
import requests
from datetime import datetime, timedelta
import json
from urllib.parse import urlencode
import boto3
import random




def refresh_token(location='CoachMcloone', token_file_path = 'app/private'):
    """
    Refreshes the authentication token. 
    Parameters:
    - location (str): The location of the business in Pascal case, i.e. 'SamLab' or 'CoachMcloone'.
    - token_file_path (str): The path to the token file relative to the directory of the lambda function

    This function does the following:
    1. Reads the application configuration from a local file.
    2. Attempts to load the authentication tokens from a local file.
    3. If local file is not found, it retrieves the tokens from an S3 bucket.
    4. Checks if the 'SamLab' token is present in the loaded tokens. If not, it returns an error.
    5. Prepares data for the token refresh request, including client_id, client_secret, grant_type, refresh_token, user_type, and redirect_uri from the application configuration and SamLab token.
    6. Sends a POST request to the Lead Connector API to refresh the token.
    7. If the token refresh request is successful, it updates the 'SamLab' token in the loaded tokens with the response and tries to save it to a local file.
    8. If local save fails, it tries to save the tokens to the same S3 bucket.
    9. If S3 save also fails, it prints the error details.
    10. Returns the response of the token refresh request.

    Returns:
        dict: A dictionary with 'statusCode', 'body', and optionally 'response' keys. 'statusCode' is 200 if the token refresh request is successful, and 500 otherwise. 'body' contains the response of the token refresh request if it is successful, and an error message otherwise. 'response' is present only if the token refresh request fails, and contains the response of the failed request.
    """

    filename = 'auth_token_response_cicd.json'
    # filename = 'auth_token_response.json' # Original
    config_file_name = 'config.json'
    # config_file_name = 'config_cicid_app.json' # cicd

    with open(f'{token_file_path}/{config_file_name}') as config_file:
        appConfig = json.load(config_file)
    s3 = boto3.client('s3')
    response = s3.get_object(Bucket='ownitfit-silvhua', Key=filename)
    tokens = json.loads(response['Body'].read().decode('utf-8'))
    if location in tokens:
        previous_token = tokens[location]
        print(f'Tokens retrieved from S3 for {location}.')
    else:
        return {
            'statusCode': 500,
            'body': json.dumps({"error": "SamLab token not found in token_file."})
        }

    data = {
        'client_id': appConfig["clientId"],
        'client_secret': appConfig["clientSecret"],
        'grant_type': 'refresh_token',
        'refresh_token': previous_token["refresh_token"],
        'user_type': 'Location',
        # 'redirect_uri': 'https://6r8pb7q836.execute-api.us-west-2.amazonaws.com/oauth/callback',
        'redirect_uri': 'http://localhost:3000/oauth/callback'
    }

    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    response = requests.post('https://services.leadconnectorhq.com/oauth/token', data=urlencode(data), headers=headers)

    if response.status_code == 200:
        tokens[location] = response.json()
        # pprint(f'Tokens: {tokens["SamLab"]}')
        try:
            # Save tokens to S3
            s3 = boto3.client('s3')
            s3.put_object(
                Body=json.dumps(tokens), 
                Bucket='ownitfit-silvhua', Key=filename
                )
            print(f'Tokens saved to S3 {location}.')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print(f"Unable to save tokens to S3. Error in line {lineno} of {filename}: {str(error)}")
        return {
            'statusCode': 200,
            'body': json.dumps(response.json())
        }
    else:
        return {
            'statusCode': 500,
            'body': json.dumps({"error": f"Failed to fetch access token: {response.reason}",}),
            'response': json.dumps(response.json())
        }

def ghl_request(
        contactId, endpoint='createTask', text=None, payload=None, location='SamLab', 
        path_param=None, locationId=None
        ):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID OR locationId if endpoint is 'getWorkflow'.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'workflow', 'getWorkflow', \
    'createNote', 'send_message', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.
    - path_param (str): Additional path parameter for the request.
    - locationId: locationId for getContacts endpoint.

    Example payload for sendMessage endpoint:
        
            payload = {
                "type": "Email",
                "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
                "subject": "Testing GHL API with Python threadId",
                "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
                "threadId": 'Y0ecBIIPiHa7bpZr616G'
            }

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'
    if payload:
        print(f'input payload: {payload}')
    try:
        if endpoint == 'getContact':
            endpoint_url = f'contacts/{contactId}'
            request_type = 'GET'
            payload = None
        elif endpoint == 'createTask':
            endpoint_url = f'contacts/{contactId}/tasks'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['title'] = f'Send message to contact {contactId}'
                payload['body'] = text if text else f"Test task via GHL API at {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')} Pacific time"
            payload['dueDate'] = payload[3] if len(payload) > 3 else datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
            payload['completed'] = False
        elif endpoint == 'workflow':
            endpoint_url = f'contacts/{contactId}/workflow/{path_param}'
            request_type = 'POST'
            payload = {}
            payload['eventStartTime'] = (datetime.utcnow() + timedelta(minutes=random.randint(2, 10))).strftime('%Y-%m-%dT%H:%M:%S+00:00')
        elif endpoint == 'getWorkflow': ### 
            endpoint_url = r'workflows/'
            request_type = 'GET'
            params = {'locationId': contactId if contactId else locationId}
        elif endpoint == 'createNote':
            endpoint_url = f'contacts/{contactId}/notes'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['body'] = (f"Reply to SMS (contactID {contactId}) {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}" if text==None else text)
                payload['userId'] = contactId
        elif endpoint == 'sendMessage':
            endpoint_url = f'conversations/messages'
            request_type = 'POST'
            payload["contactId"] = contactId
            if payload['type'] == 'Email':
                payload['html'] = payload['message']
        elif endpoint == 'getEmailHistory':
            endpoint_url = f'conversations/search?contactId={contactId}'
            request_type = 'GET'
            payload = None
        elif endpoint == 'getContacts':  
            endpoint_url = f'contacts/'
            request_type = 'GET'
            payload = None
            params = {
                'locationId': locationId,
                'query': contactId
            }
        else:
            raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'sendMessage', and 'getEmailHistory'.")

        url = f'{url_root}{endpoint_url}'
        token_filename = 'auth_token_response_cicd.json'
        try:
            s3 = boto3.client('s3')
            response = s3.get_object(Bucket='ownitfit-silvhua', Key=token_filename)
            token = json.loads(response['Body'].read().decode('utf-8'))[location]
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            message = f'Error in line {lineno} of {filename}: {str(error)}'
            print(message)
            return message

        headers = {
            "Authorization": f"Bearer {token['access_token']}",
            "Version": "2021-07-28",
            "Content-Type": "application/json",
            "Accept": "application/json"
        }


        if request_type == 'POST':
            response = requests.post(
                url, headers=headers, 
                json=payload if payload else None
            )
        elif request_type == 'GET':
            response = requests.get(
                url, headers=headers, 
                json=payload if payload else None,
                params=params if params else None
            )
        else:
            raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

        print(f'Status code {response.status_code}: {response.reason}')
        data = response.json()
        data['status_code'] = response.status_code
        data['response_reason'] = response.reason
        try:
            if endpoint == 'getEmailHistory':
                email_timestamp = data['conversations'][0]['dateUpdated']/1000
                utc_time = datetime.utcfromtimestamp(email_timestamp)
                pacific_time = utc_time.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('US/Pacific'))
                email_timestamp_str = pacific_time.strftime('%Y-%m-%d %H:%M:%S')

                print(f'Last email sent: {email_timestamp_str} Pacific time')
                data['emailTimestamp_pacific'] = email_timestamp_str
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print(f'Error in line {lineno} of {filename}: {str(error)}')
            return '[Chatbot response]'
        return data
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print(f'Error in line {lineno} of {filename}: {str(error)}')
        return '[Chatbot response]'

iteration = 2.3
query_string = leads_df.loc[1533, 'Name']

# token_response = refresh_token('CoachMcloone', '../src/app/private')

response_dict[iteration] = ghl_request(
    query_string, endpoint='getContacts', text=None, payload=None, location='CoachMcloone', 
    path_param=None, locationId=locationId
)

Status code 200: OK


In [28]:
response_dict[iteration]

{'contacts': [{'id': '51b7JClFlLkzLdrUjtJs',
   'locationId': 'fsdVH26v30hoBBQOBttG',
   'contactName': 'mary daughton',
   'firstName': 'mary',
   'lastName': 'daughton',
   'companyName': None,
   'email': 'maryhealy507@gmail.com',
   'phone': '+353872726049',
   'dnd': False,
   'type': 'lead',
   'source': 'facebook form lead',
   'assignedTo': None,
   'city': None,
   'state': None,
   'postalCode': None,
   'address1': None,
   'dateAdded': '2023-12-13T08:53:51.318Z',
   'dateUpdated': '2023-12-29T18:33:05.631Z',
   'dateOfBirth': None,
   'businessId': None,
   'tags': ['money_magnet_schedule', 'money_magnet_lead'],
   'followers': [],
   'additionalEmails': [],
   'attributions': [{'utmSessionSource': 'Other',
     'isFirst': True,
     'medium': 'facebook',
     'mediumId': '24317394204571380'}],
   'country': 'IE',
   'timezone': 'Europe/London',
   'customFields': []}],
 'meta': {'total': 1,
  'nextPageUrl': 'http://services.leadconnectorhq.com/contacts/?locationId=fsdVH26v

### Make sure `getWorkflow` endpoint works

In [31]:
response_dict[iteration] = ghl_request(
    auth_info['SamLab']['locationId'], endpoint='getWorkflow', text=None, payload=None, 
    path_param=None
)

Status code 200: OK


# Get contact ID 

In [33]:
iteration = 2.31
query_string = leads_df.loc[1533, 'Name']

# token_response = refresh_token('CoachMcloone', '../src/app/private')

response_dict[iteration] = ghl_request(
    query_string, endpoint='getContacts', text=None, payload=None, location='CoachMcloone', 
    path_param=None, locationId=locationId
)

Status code 200: OK


In [34]:
response_dict[iteration]

{'contacts': [{'id': '51b7JClFlLkzLdrUjtJs',
   'locationId': 'fsdVH26v30hoBBQOBttG',
   'contactName': 'mary daughton',
   'firstName': 'mary',
   'lastName': 'daughton',
   'companyName': None,
   'email': 'maryhealy507@gmail.com',
   'phone': '+353872726049',
   'dnd': False,
   'type': 'lead',
   'source': 'facebook form lead',
   'assignedTo': None,
   'city': None,
   'state': None,
   'postalCode': None,
   'address1': None,
   'dateAdded': '2023-12-13T08:53:51.318Z',
   'dateUpdated': '2023-12-29T18:33:05.631Z',
   'dateOfBirth': None,
   'businessId': None,
   'tags': ['money_magnet_schedule', 'money_magnet_lead'],
   'followers': [],
   'additionalEmails': [],
   'attributions': [{'utmSessionSource': 'Other',
     'isFirst': True,
     'medium': 'facebook',
     'mediumId': '24317394204571380'}],
   'country': 'IE',
   'timezone': 'Europe/London',
   'customFields': []}],
 'meta': {'total': 1,
  'nextPageUrl': 'http://services.leadconnectorhq.com/contacts/?locationId=fsdVH26v

# Parse contactId

In [36]:
contactId_dict = dict()

In [38]:
def parse_contactId(getContacts_response):
    first_result = getContacts_response['contacts'][0]
    # print(f'contactId for {first_result["contactName"]}')
    return first_result['id']

contact = 1
contactId_dict[contact] = parse_contactId(response_dict[iteration])

# Get conversation

In [40]:
# import pytz # Not included in Lambda by default
import sys
import requests
from datetime import datetime, timedelta
import json
from urllib.parse import urlencode
import boto3
import random




def refresh_token(location='CoachMcloone', token_file_path = 'app/private'):
    """
    Refreshes the authentication token. 
    Parameters:
    - location (str): The location of the business in Pascal case, i.e. 'SamLab' or 'CoachMcloone'.
    - token_file_path (str): The path to the token file relative to the directory of the lambda function

    This function does the following:
    1. Reads the application configuration from a local file.
    2. Attempts to load the authentication tokens from a local file.
    3. If local file is not found, it retrieves the tokens from an S3 bucket.
    4. Checks if the 'SamLab' token is present in the loaded tokens. If not, it returns an error.
    5. Prepares data for the token refresh request, including client_id, client_secret, grant_type, refresh_token, user_type, and redirect_uri from the application configuration and SamLab token.
    6. Sends a POST request to the Lead Connector API to refresh the token.
    7. If the token refresh request is successful, it updates the 'SamLab' token in the loaded tokens with the response and tries to save it to a local file.
    8. If local save fails, it tries to save the tokens to the same S3 bucket.
    9. If S3 save also fails, it prints the error details.
    10. Returns the response of the token refresh request.

    Returns:
        dict: A dictionary with 'statusCode', 'body', and optionally 'response' keys. 'statusCode' is 200 if the token refresh request is successful, and 500 otherwise. 'body' contains the response of the token refresh request if it is successful, and an error message otherwise. 'response' is present only if the token refresh request fails, and contains the response of the failed request.
    """

    filename = 'auth_token_response_cicd.json'
    # filename = 'auth_token_response.json' # Original
    config_file_name = 'config.json'
    # config_file_name = 'config_cicid_app.json' # cicd

    with open(f'{token_file_path}/{config_file_name}') as config_file:
        appConfig = json.load(config_file)
    s3 = boto3.client('s3')
    response = s3.get_object(Bucket='ownitfit-silvhua', Key=filename)
    tokens = json.loads(response['Body'].read().decode('utf-8'))
    if location in tokens:
        previous_token = tokens[location]
        print(f'Tokens retrieved from S3 for {location}.')
    else:
        return {
            'statusCode': 500,
            'body': json.dumps({"error": "SamLab token not found in token_file."})
        }

    data = {
        'client_id': appConfig["clientId"],
        'client_secret': appConfig["clientSecret"],
        'grant_type': 'refresh_token',
        'refresh_token': previous_token["refresh_token"],
        'user_type': 'Location',
        # 'redirect_uri': 'https://6r8pb7q836.execute-api.us-west-2.amazonaws.com/oauth/callback',
        'redirect_uri': 'http://localhost:3000/oauth/callback'
    }

    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    response = requests.post('https://services.leadconnectorhq.com/oauth/token', data=urlencode(data), headers=headers)

    if response.status_code == 200:
        tokens[location] = response.json()
        # pprint(f'Tokens: {tokens["SamLab"]}')
        try:
            # Save tokens to S3
            s3 = boto3.client('s3')
            s3.put_object(
                Body=json.dumps(tokens), 
                Bucket='ownitfit-silvhua', Key=filename
                )
            print(f'Tokens saved to S3 {location}.')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print(f"Unable to save tokens to S3. Error in line {lineno} of {filename}: {str(error)}")
        return {
            'statusCode': 200,
            'body': json.dumps(response.json())
        }
    else:
        return {
            'statusCode': 500,
            'body': json.dumps({"error": f"Failed to fetch access token: {response.reason}",}),
            'response': json.dumps(response.json())
        }

def ghl_request(
        contactId, endpoint='createTask', text=None, payload=None, location='SamLab', 
        path_param=None, locationId=None, params_dict=None
        ):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID OR locationId if endpoint is 'getWorkflow'.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'workflow', 'getWorkflow', \
    'createNote', 'send_message', 'getContacts', 'searchConversations', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.
    - path_param (str): Additional path parameter for the request.
    - locationId: locationId for getContacts endpoint.

    Example payload for sendMessage endpoint:
        
            payload = {
                "type": "Email",
                "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
                "subject": "Testing GHL API with Python threadId",
                "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
                "threadId": 'Y0ecBIIPiHa7bpZr616G'
            }

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'
    if payload:
        print(f'input payload: {payload}')
    try:
        if endpoint == 'getContact':
            endpoint_url = f'contacts/{contactId}'
            request_type = 'GET'
            payload = None
        elif endpoint == 'createTask':
            endpoint_url = f'contacts/{contactId}/tasks'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['title'] = f'Send message to contact {contactId}'
                payload['body'] = text if text else f"Test task via GHL API at {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')} Pacific time"
            payload['dueDate'] = payload[3] if len(payload) > 3 else datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
            payload['completed'] = False
        elif endpoint == 'workflow':
            endpoint_url = f'contacts/{contactId}/workflow/{path_param}'
            request_type = 'POST'
            payload = {}
            payload['eventStartTime'] = (datetime.utcnow() + timedelta(minutes=random.randint(2, 10))).strftime('%Y-%m-%dT%H:%M:%S+00:00')
        elif endpoint == 'getWorkflow': ### 
            endpoint_url = r'workflows/'
            request_type = 'GET'
            params = {'locationId': contactId if contactId else locationId}
        elif endpoint == 'createNote':
            endpoint_url = f'contacts/{contactId}/notes'
            request_type = 'POST'
            if payload == None:
                payload = {}
                payload['body'] = (f"Reply to SMS (contactID {contactId}) {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}" if text==None else text)
                payload['userId'] = contactId
        elif endpoint == 'sendMessage':
            endpoint_url = f'conversations/messages'
            request_type = 'POST'
            payload["contactId"] = contactId
            if payload['type'] == 'Email':
                payload['html'] = payload['message']
        elif endpoint == 'getEmailHistory':
            endpoint_url = f'conversations/search?contactId={contactId}'
            request_type = 'GET'
            payload = None
        elif (endpoint == 'getContacts') | (endpoint == 'searchConversations'):  
            endpoint_url = f'conversations/search' if endpoint=='searchConversations' else f'contacts/' 
            request_type = 'GET'
            payload = None
            if params_dict:
                params = params_dict
            else:
                params = {
                    'locationId': locationId,
                    'query': contactId
                }
        else:
            raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'sendMessage', and 'getEmailHistory'.")

        url = f'{url_root}{endpoint_url}'
        token_filename = 'auth_token_response_cicd.json'
        try:
            s3 = boto3.client('s3')
            response = s3.get_object(Bucket='ownitfit-silvhua', Key=token_filename)
            token = json.loads(response['Body'].read().decode('utf-8'))[location]
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            message = f'Error in line {lineno} of {filename}: {str(error)}'
            print(message)
            return message

        headers = {
            "Authorization": f"Bearer {token['access_token']}",
            "Version": "2021-07-28",
            "Content-Type": "application/json",
            "Accept": "application/json"
        }


        if request_type == 'POST':
            response = requests.post(
                url, headers=headers, 
                json=payload if payload else None
            )
        elif request_type == 'GET':
            response = requests.get(
                url, headers=headers, 
                json=payload if payload else None,
                params=params if params else None
            )
        else:
            raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

        print(f'Status code {response.status_code}: {response.reason}')
        data = response.json()
        data['status_code'] = response.status_code
        data['response_reason'] = response.reason
        try:
            if endpoint == 'getEmailHistory':
                email_timestamp = data['conversations'][0]['dateUpdated']/1000
                utc_time = datetime.utcfromtimestamp(email_timestamp)
                pacific_time = utc_time.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('US/Pacific'))
                email_timestamp_str = pacific_time.strftime('%Y-%m-%d %H:%M:%S')

                print(f'Last email sent: {email_timestamp_str} Pacific time')
                data['emailTimestamp_pacific'] = email_timestamp_str
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print(f'Error in line {lineno} of {filename}: {str(error)}')
            return '[Chatbot response]'
        return data
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print(f'Error in line {lineno} of {filename}: {str(error)}')
        return '[Chatbot response]'

iteration = 3
query_string = leads_df.loc[1533, 'Name']

# token_response = refresh_token('CoachMcloone', '../src/app/private')

response_dict[iteration] = ghl_request(
    query_string, endpoint='searchConversations', text=None, payload=None, location='CoachMcloone', 
    path_param=None, locationId=locationId
)

Status code 200: OK


In [41]:
response_dict[iteration]

{'conversations': [{'id': 'u9NVqZG3g3IyBmt9j5tu',
   'locationId': 'fsdVH26v30hoBBQOBttG',
   'dateAdded': 1702457631404,
   'dateUpdated': 1703874784846,
   'lastMessageDate': 1703874784845,
   'lastMessageType': 'TYPE_SMS',
   'lastMessageBody': 'Hi Mary  our call is coming up in 2 hours! Excited to hear about you and your situation!',
   'lastOutboundMessageAction': 'automated',
   'lastMessageDirection': 'outbound',
   'inbox': True,
   'unreadCount': 0,
   'lastManualMessageDate': 1703794053248,
   'contactId': '51b7JClFlLkzLdrUjtJs',
   'fullName': 'Mary Daughton',
   'contactName': 'Mary Daughton',
   'profilePhoto': 'https://platform-lookaside.fbsbx.com/platform/profilepic/?eai=AXG142FWa--BKZdmZVjO52Zqryq1-YCNAlEY7l1qLSImKvfjN5kHd-fTNqxmbIA2pXE5-q2mBmnP&psid=24317394204571380&width=1024&ext=1706385823&hash=Afpxh6d2K1N5S--zjNWRdSthXJgPy_yQuc4NQGZI3SS6tA',
   'email': 'maryhealy507@gmail.com',
   'phone': '+353872726049',
   'tags': ['money_magnet_schedule', 'money_magnet_lead'],

## Parse result ID

In [45]:
conversation_id_dict = dict()

In [46]:
def parse_result_id(response, result_type):
    """
    Parse the GHL API response and return the `id` of the first result.
    
    Parameters:
    - response (dict): GHL API response.
    - result_type (str): Value is dependent on the GHL API endpoint:
        - 'searchConversations' endpoint --> result_type='conversations
        - 'getContacts' endpoint --> result_type='contacts'

    Returns: `id` of the first result    
    """
    first_result = response[result_type][0]
    return first_result.get('id', 'Unable to parse id')

contact = 1
conversation_id_dict[contact] = parse_result_id(response_dict[iteration], 'conversations')

# *End of Page*